# Proyecto #2 - Security Data Science

María Isabel Ortiz Naranjo -
Luis Alejandro Urbina Hernández - 18473

Importación de librerías que utilizaremos a lo largo del desarrollo del proyecto

In [3]:
import numpy as np
import pandas as pd

## Parte 1: Sampling de datos

El primer problema que encontramos fue el tamaño de los datasets de entreno y pruebas proporcionados por Microsoft. Por lo que se decidió utilizar una muestra pequeña pero significativa de dichos datos. En concreto, se decidió utilizar 1/256 de datos aleatorios del set de entrenamiento (train.csv) que originalmente contenía 8,921,483 datos y 1/256 de datos aleatorios del set de testeo (test.csv) que originalmente contenía 7,853,253 datos.

### Declaración de los tipos que se utilizarán en la data

In [2]:
my_data_types = {
    "MachineIdentifier":                                    "category",
    "ProductName":                                          "category",
    "EngineVersion":                                        "category",
    "AppVersion":                                           "category",
    "AvSigVersion":                                         "category",
    "IsBeta":                                               "int8",
    "RtpStateBitfield":                                     "float16",
    "IsSxsPassiveMode":                                     "int8",
    "DefaultBrowsersIdentifier":                            "float32",
    "AVProductStatesIdentifier":                            "float32",
    "AVProductsInstalled":                                  "float16",
    "AVProductsEnabled":                                    "float16",
    "HasTpm":                                               "int",
    "CountryIdentifier":                                    "int16",
    "CityIdentifier":                                       "float32",
    "OrganizationIdentifier":                               "float16",
    "GeoNameIdentifier":                                    "float16",
    "LocaleEnglishNameIdentifier":                          "int16",
    "Platform":                                             "category",
    "Processor":                                            "category",
    "OsVer":                                                "category",
    "OsBuild":                                              "int16",
    "OsSuite":                                              "int16",
    "OsPlatformSubRelease":                                 "category",
    "OsBuildLab":                                           "category",
    "SkuEdition":                                           "category",
    "IsProtected":                                          "float16",
    "AutoSampleOptIn":                                      "int8",
    "PuaMode":                                              "category",
    "SMode":                                                "float16",
    "IeVerIdentifier":                                      "float16",
    "SmartScreen":                                          "category",
    "Firewall":                                             "float16",
    "Census_MDC2FormFactor":                                "category",
    "Census_DeviceFamily":                                  "category",
    "Census_OEMNameIdentifier":                             "float32",
    "Census_OEMModelIdentifier":                            "float32",
    "Census_ProcessorCoreCount":                            "float16",
    "Census_ProcessorManufacturerIdentifier":               "float16",
    "Census_ProcessorModelIdentifier":                      "float32",
    "Census_ProcessorClass":                                "category",
    "Census_PrimaryDiskTotalCapacity":                      "float64",
    "Census_PrimaryDiskTypeName":                           "category",
    "Census_SystemVolumeTotalCapacity":                     "float64",
    "Census_HasOpticalDiskDrive":                           "int8",
    "Census_TotalPhysicalRAM":                              "float32",
    "Census_ChassisTypeName":                               "category",
    "Census_InternalPrimaryDiagonalDisplaySizeInInches":    "float32",
    "Census_InternalPrimaryDisplayResolutionHorizontal":    "float32",
    "Census_InternalPrimaryDisplayResolutionVertical":      "float32",
    "Census_PowerPlatformRoleName":                         "category",
    "Census_InternalBatteryType":                           "category",
    "Census_InternalBatteryNumberOfCharges":                "float64",
    "Census_OSVersion":                                     "category",
    "Census_OSArchitecture":                                "category",
    "Census_OSBranch":                                      "category",
    "Census_OSBuildNumber":                                 "int16",
    "Census_OSBuildRevision":                               "int32",
    "Census_OSEdition":                                     "category",
    "Census_OSSkuName":                                     "category",
    "Census_OSInstallTypeName":                             "category",
    "Census_OSInstallLanguageIdentifier":                   "float16",
    "Census_OSUILocaleIdentifier":                          "int16",
    "Census_OSWUAutoUpdateOptionsName":                     "category",
    "Census_IsPortableOperatingSystem":                     "int8",
    "Census_GenuineStateName":                              "category",
    "Census_ActivationChannel":                             "category",
    "Census_IsFlightingInternal":                           "float16",
    "Census_IsFlightsDisabled":                             "float16",
    "Census_FlightRing":                                    "category",
    "Census_ThresholdOptIn":                                "float16",
    "Census_FirmwareManufacturerIdentifier":                "float16",
    "Census_FirmwareVersionIdentifier":                     "float32",
    "Census_IsSecureBootEnabled":                           "int8",
    "Census_IsWIMBootEnabled":                              "float16",
    "Census_IsVirtualDevice":                               "float16",
    "Census_IsTouchEnabled":                                "int8",
    "Census_IsPenCapable":                                  "int8",
    "Census_IsAlwaysOnAlwaysConnectedCapable":              "float16",
    "Wdft_IsGamer":                                         "float16",
    "Wdft_RegionIdentifier":                                "float16",
    "HasDetections":                                        "float16"
}

### Importamos los CSVs con la data y los convertimos en dataframes (en cells separadas para evitar que el programa se trabe)

In [4]:
testDF = pd.read_csv('test.csv', dtype=my_data_types)

In [5]:
trainDF = pd.read_csv('train.csv', dtype=my_data_types)

### Sacamos los samples de cada data frame

In [6]:
# Sacar un sample de 1/256 del tamaño del dataset original

sampleTestDF = testDF.sample(n = int(testDF.shape[0]/256))

In [7]:
# Imprimir dimensiones del sample dataFrame

sampleTestDF.shape

(30676, 82)

In [8]:
# Imprimir cabeza del sample dataFrame

sampleTestDF.head()

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier
5762678,bbd443b47f4d84dcacf2a23d6553ba2c,win8defender,1.1.15400.5,4.18.1810.5,1.281.90.0,0,7.0,0,NaN,53447.0,...,628.0,47409.0,0,NaN,0.0,0,0,0.0,0.0,3.0
3154936,66cc25057141a28bf9c3808c150dd73f,win8defender,1.1.15300.6,4.18.1809.2,1.277.832.0,0,7.0,0,NaN,53447.0,...,554.0,33066.0,1,NaN,0.0,0,0,0.0,1.0,4.0
2301924,4b0639733251cdb8c719dbf2041bf364,win8defender,1.1.15300.6,4.18.1809.2,1.277.515.0,0,7.0,0,NaN,53447.0,...,897.0,7314.0,1,NaN,0.0,0,0,0.0,0.0,13.0
6034363,c4b8d3eac44e254bd5005eb7dd9c7e1c,win8defender,1.1.15400.4,4.18.1810.5,1.279.516.0,0,7.0,0,NaN,53447.0,...,142.0,70418.0,1,0.0,0.0,0,0,0.0,0.0,15.0
7287501,ed8a91f7430b5131c71d6a41d0ec8985,win8defender,1.1.15300.6,4.18.1809.2,1.277.911.0,0,0.0,0,NaN,53447.0,...,556.0,56929.0,0,0.0,0.0,0,0,0.0,0.0,10.0


In [9]:
# Agrupamos el dataset anterior por los dos posibles valores de HasDetections

trainDFGrouped = trainDF.groupby('HasDetections', group_keys=False)

In [10]:
# Sacamos un sample de 1/256 del tamaño del dataset original

sampleTrainDF = trainDFGrouped.apply(lambda x: x.sample(min(len(x), int(trainDF.shape[0]/256))))

In [11]:
# Imprimir dimensiones del sample dataFrame

sampleTrainDF.shape

(69698, 83)

In [12]:
# Imprimir cabeza del sample dataFrame

sampleTrainDF.head()

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
5988857,abe3529fa18560122e75e2089dbf80eb,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1863.0,0,7.0,0,NaN,53447.0,...,33039.0,1,NaN,0.0,0,0,0.0,0.0,15.0,0.0
2833233,514c8406cb449e5e300b483418d1f2a4,win8defender,1.1.15100.1,4.16.17656.18052,1.273.1208.0,0,7.0,0,NaN,53447.0,...,48453.0,0,NaN,0.0,0,0,0.0,0.0,11.0,0.0
6720797,c0e0d3782523a9a7755d642721ca1df8,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1270.0,0,7.0,0,NaN,53447.0,...,8126.0,0,NaN,1.0,0,1,0.0,0.0,7.0,0.0
477576,0dbbbd704bb660e7a9b093b124ba7f4b,win8defender,1.1.15200.1,4.18.1807.18075,1.275.938.0,0,7.0,0,NaN,53447.0,...,13299.0,0,0.0,1.0,0,0,0.0,0.0,13.0,0.0
2154397,3dd376626fa97394523d18184276721a,win8defender,1.1.14901.4,4.16.17656.18052,1.269.913.0,0,7.0,0,NaN,62773.0,...,33157.0,0,0.0,0.0,0,0,0.0,0.0,11.0,0.0


### Guardar los samples en el directorio actual como CSVs (estos servirán para que a la hora de probarlo no sea necesario volver a ejecutar el proceso sampling desde cero, por cuestiones de tiempo)

In [13]:
sampleTestDF.to_csv('sample_test.csv')

In [14]:
sampleTrainDF.to_csv('sample_train.csv')